In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
from model import MukkeBudeLSTM
from mapping import MusicMapping
import utils
import music21 as m21
import tensorflow as tf
import numpy as np
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [4]:
# Load songs
paths = m21.corpus.getComposer('bach')
paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = utils.load_dataset_lstm(paths, 64)

print(f"Songs encoded: {len(encoded_songs)}")

Found 1 songs in corpus.
Removed 0 bad songs
Remaining songs: 1
Songs encoded: 2


In [5]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)
model.train(encoded_songs)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 298)]       0         
                                                                 
 lstm (LSTM)                 (None, 256)               568320    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 298)               76586     
                                                                 
Total params: 644,906
Trainable params: 644,906
Non-trainable params: 0
_________________________________________________________________
26/26 [==============================] - 7s 160ms/step - loss: 2.7939 - accuracy: 0.6226


In [6]:
# Create song
generated_song = model.generate("n72 _ _ _ _ _ n72 _ _ _ _ _ n72 _ n71 _")
new_song = utils.decode_songs_old(generated_song)

1/1 [==============================] - 0s 22ms/step


In [7]:
path = Path("generated_song_bach.mid")
utils.write_midi(new_song, path)
utils.write_musicxml(new_song, "generated_song_bach.musicxml")

In [ ]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}